# Code-to-Code Comparison: IEA Task 26

National Renewable Energy Laboratory\
Rob Hammond\
1 July 2024

**Note**: As of v0.7, 100% reduction examples are no longer advised to be considered due to the more accurate downtime modeling, that render them invalid. With the full extent of bug fixes in v0.7, the availability is in the range of 0%-1%.

In [1]:
from copy import deepcopy
from time import perf_counter

import yaml
import pandas as pd

from wombat import Simulation
from wombat.core.library import IEA_26

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = "{:,.2f}".format

In [2]:
tech_salary_annual = 100000
techs = 30
capacity = 400 * 1000  # 400MW -> kW
tech_salary_annual * techs / capacity

7.5

In [3]:
configs = [
    "requests",
    "one_mobilization",
    "two_mobilizations",
    "three_mobilizations",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "total annual costs": [],
    "technicians": [],
    "materials": [],
    "vessels": [],
    "ctv cost": [],
    "hlv cost": [],
    "dsv cost": [],
    "cab cost": [],
    "manual reset": [],
    "minor repair": [],
    "major repair": [],
    "major replacement": [],
    "remote reset": [],
    "annual service": [],
    "bos": [],  #  substructure inspection + scour repair + substation inspection + small/large transformer repairs
    "total downtime": [],
    "ctv utilization": [],
    "hlv utilization": [],
    "dsv utilization": [],
    "cab utilization": [],
}
metrics_dict = {}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(IEA_26, f"{config}.yaml", random_seed=2023)
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(45)} | {(end - start) / 60:.2f} m")
    metrics_dict[config] = sim.metrics

                              iea_26_requests | 1.19 m
                      iea_26_one_mobilization | 1.11 m
                     iea_26_two_mobilizations | 1.11 m
                   iea_26_three_mobilizations | 1.11 m


In [5]:
for config, metrics in metrics_dict.items():
    # Gather the results of interest
    years = metrics.events.year.unique().shape[0]
    mil = 1000000

    availability = metrics.time_based_availability(
        frequency="project", by="windfarm"
    ).values[0][0]
    availability_production = metrics.production_based_availability(
        frequency="project", by="windfarm"
    ).values[0][0]
    cf_net = metrics.capacity_factor(
        which="net", frequency="project", by="windfarm"
    ).values[0][0]
    cf_gross = metrics.capacity_factor(
        which="gross", frequency="project", by="windfarm"
    ).values[0][0]
    power_production = metrics.power_production(
        frequency="project", by="windfarm"
    ).values[0][0]
    completion_rate = metrics.task_completion_rate(
        which="both", frequency="project"
    ).values[0][0]
    parts = metrics.events[["materials_cost"]].sum().sum()
    techs = metrics.project_fixed_costs(frequency="project", resolution="medium").labor[
        0
    ]
    total = metrics.events[["total_cost"]].sum().sum()

    equipment = metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    ctv = (
        equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]]
        .sum()
        .sum()
    )
    hlv = (
        equipment[[el for el in equipment.columns if "Jack-up Vessel" in el]]
        .sum()
        .sum()
    )
    dsv = (
        equipment[[el for el in equipment.columns if "Diving Support Vessel" in el]]
        .sum()
        .sum()
    )
    cab = (
        equipment[[el for el in equipment.columns if "Cable Laying Vessel" in el]]
        .sum()
        .sum()
    )

    times = metrics.process_times()
    times = times / years / 24 / 100  # events per turbine and year

    utilization = metrics.service_equipment_utilization(frequency="project")
    ctv_ur = (
        utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]]
        .mean()
        .mean()
    )
    hlv_ur = (
        utilization[[el for el in utilization.columns if "Jack-up Vessel" in el]]
        .mean()
        .mean()
    )
    dsv_ur = (
        utilization[[el for el in utilization.columns if "Diving Support Vessel" in el]]
        .mean()
        .mean()
    )
    cab_ur = (
        utilization[[el for el in utilization.columns if "Cable Laying Vessel" in el]]
        .mean()
        .mean()
    )

    # Log the results of interest
    results["availability - time based"].append(availability)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["total annual costs"].append((total + techs) / mil / years)
    results["technicians"].append(techs / mil / years)
    results["materials"].append(parts / mil / years)
    results["vessels"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["dsv cost"].append(dsv / mil / years)
    results["cab cost"].append(cab / mil / years)
    results["manual reset"].append(
        times.loc[times.index.intersection(["manual reset"]), "downtime"].sum()
    )
    results["minor repair"].append(
        times.loc[
            times.index.intersection(
                [
                    "minor repair",
                ]
            ),
            "downtime",
        ].sum()
    )
    results["major repair"].append(
        times.loc[times.index.intersection(["major repair"]), "downtime"].sum()
    )
    results["major replacement"].append(
        times.loc[times.index.intersection(["major replacement"]), "downtime"].sum()
    )
    results["remote reset"].append(
        times.loc[times.index.intersection(["remote reset"]), "downtime"].sum()
    )
    results["annual service"].append(
        times.loc[times.index.intersection(["annual service"]), "downtime"].sum()
    )
    ix = [
        "substructure inspection",
        "substation inspection",
        "small foundation/scour repair",
        "cable replacement",
        "small transformer repair",
        "large transformer repair",
    ]
    results["bos"].append(times.loc[times.index.intersection(ix), "downtime"].sum())
    results["total downtime"].append(times.loc[:, "downtime"].sum())
    results["ctv utilization"].append(ctv_ur)
    results["hlv utilization"].append(hlv_ur)
    results["dsv utilization"].append(dsv_ur)
    results["cab utilization"].append(cab_ur)

In [6]:
# Save the results
# pickled dictionary format
with open(IEA_26 / "results" / "results_dict_v0.9.4.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(
    results.values(), columns=columns, index=results.keys()
).fillna(0)
results_df.to_csv(IEA_26 / "results" / "results_data_v0.9.4.csv", index_label="result")

In [7]:
results_df

,requests,one_mobilization,two_mobilizations,three_mobilizations
availability - time based,0.97,0.97,0.97,0.97
availability - production based,0.97,0.97,0.97,0.97
capacity factor - net,0.57,0.57,0.57,0.57
capacity factor - gross,0.59,0.59,0.59,0.59
power production,"39,878.41","40,093.56","40,048.55","40,091.78"
task completion rate,0.94,0.94,0.94,0.94
total annual costs,21.84,17.17,21.80,24.96
technicians,3.00,3.00,3.00,3.00
materials,7.20,5.96,7.26,7.14
vessels,11.63,8.22,11.53,14.82
